In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import json
import time

In [2]:
baseUrl = "https://gov.optimism.io"

In [1]:
def scrapeFirstCategories(baseUrl):
    driver = webdriver.Chrome()
    driver.get(baseUrl)
    soup = BeautifulSoup(driver.page_source,"html.parser")
    driver.quit()
    container = soup.find("div",class_="custom-category-boxes-container")
    list = container.find_all("a",class_="parent-box-link")
    categories = {}
    for i in list:
        categories[i.text.replace("\n","").strip()] = baseUrl+i.attrs["href"]

    return categories,baseUrl

def scrapeSecondTopics(categories,baseUrl):
    topics = {}
    for category,link in categories.items():
        driver = webdriver.Chrome()
        driver.get(link)

        lastHeight = driver.execute_script("return document.body.scrollHeight")

        for i in range(100):
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            time.sleep(2)
            currentHeight = driver.execute_script("return document.body.scrollHeight")
            if lastHeight == currentHeight:
                break

        soup = BeautifulSoup(driver.page_source,"html.parser")
    
        driver.quit()

        table = soup.find("table",class_="topic-list")
        allA = [i.a for i in table.find_all("span",class_="link-top-line")]
        topicsLinks = {}
        for i in allA :
            topicsLinks[i.text] = baseUrl+i.attrs["href"]

        topics[category] = topicsLinks
        
    return topics,baseUrl

def scrapeThirdContent(topics,baseUrl):
    main_content = {}
    backlog_scraped_2 ={}

    for category,items in topics.items():

        topics = {}
        backlog_topics_2 = {}

        for topic , link in items.items():
            try:
                driver = webdriver.Chrome()
                driver.get(link)
                lastHeight = driver.execute_script("return document.body.scrollHeight")
                for i in range (100):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(1)
                    currentHeight = driver.execute_script("return document.body.scrollHeight")
                    if currentHeight == lastHeight:
                        break
                    lastHeight = currentHeight
                

                soup = BeautifulSoup(driver.page_source,"html.parser")
                driver.quit()
                
                contents = [i.text.replace("\n","").strip() for i in soup.findAll("div",class_="cooked")]

                content = {}
                content["content"] = contents[0]
                content["comments"] = contents[1:]

                topics[topic]=content
            
            except:
                backlog_topics_2[topic]=link
                pass 
            
        main_content[category] = topics
        if len(backlog_topics_2) :
            backlog_scraped_2[category]=backlog_topics_2
    
    return main_content , backlog_scraped_2 , baseUrl

        


        

            


In [ ]:
topics = {}
with open("topics.json","r") as filer:
    topics = json.load(filer)

content = scrapeThirdContent(topics,baseUrl)